# Libraries

In [5]:
!pip install unidecode
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 8.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.1/653.1 kB 6.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.6/770.6 kB 9.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21/21 [spacy]m20/21 [spacy]


In [6]:
import pandas as pd
import re #regex expressions
import ast #for tag list
import unidecode # replacement of accented characters
import spacy

# Stage 1: Enhanced Data Cleaning, Preprocessing, and Exploratory Analysis

## 1.1 Data Collection & Cleaning

*Done by: Marisa  - Checked by: Alla*

### Data Import


- In your Google Drive, create a folder "CLT" and upload the csv with its original name (downloaded from Kaggle).

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# csv_path = '/content/drive/My Drive/CLT/ai_media_dataset_20250911.csv'
csv_path = 'data/ai_media_dataset_20250911.csv'
df = pd.read_csv(csv_path)

df.head()

,Unnamed: 0,title,date,content,domain,url,tags
0,88571,Ricoh to provide customer support for Agility ...,2024-09-11,['The Digit humanoid could work in distributio...,therobotreport,https://www.therobotreport.com/ricoh-provides-...,"['Robotics', 'Video', 'BostonDynamics']"
1,92022,MTV VMAs 2024: Live shopping is coming to the ...,2024-09-11,"['In this article', ""When viewers tune in to t...",cnbc,https://www.cnbc.com/2024/09/11/mtv-vmas-2024-...,['GenerativeAI']
2,81522,Open-source imagery is transforming investigat...,2024-09-11,['Open-source online imagery can play a vital ...,theconversation,https://www.theconversation.com/open-source-im...,"['Disinformation', 'Video', 'Deepfake', 'Infor..."
3,81521,"With China seeking AI dominance, Taiwan’ s eff...",2024-09-11,"['Tensions between China, Taiwan and the U.S. ...",theconversation,https://www.theconversation.com/with-china-see...,"['MilitaryAndDefense', 'Missiles', 'HighPerfor..."
4,63808,Study links EV charging stations to increased ...,2024-09-11,['Countries globally are rapidly transitioning...,phys,https://phys.org/news/2024-09-links-ev-station...,"['Causality', 'Planning']"


In [9]:
df.shape

(16527, 7)

The dataset contains 16'527 observations with 7 columns.

### Removing Duplicates

In [10]:
duplicate_rows = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

title_duplicate = df[df.duplicated(subset='title')]
print(f"Number of duplicate titles: {len(title_duplicate)}")

content_duplicate = df[df.duplicated(subset ='content')]
print(f"Number of duplicate content:{len(content_duplicate)}")

url_duplicate = df[df.duplicated(subset ='url')]
print(f"Number of duplicate URLs: {len(url_duplicate)}")

Number of duplicate rows: 0
Number of duplicate titles: 0
Number of duplicate content:9
Number of duplicate URLs: 0


There are no duplicate rows, titles and urls. But there is duplicate content. We take a closer look at the duplicate content:

In [11]:
duplicate_content_rows = df[df.duplicated(subset=['content'], keep=False)].sort_values(by='content')
print(duplicate_content_rows)

       Unnamed: 0                                              title  \
920         93651  Performance, Security FOMO Could Fuel The AI P...   
933         93649  A Performance, Security FOMO Could Fuel The AI...   
7070       105445  ProMat Show Planner: 2025 to be bigger and bet...   
6354       105429  ProMat 2025 show planner: Bigger and better th...   
5174       105395    Maximizing automated systems at Outdoor Network   
2582       105328           Maximizing automation at Outdoor Network   
9337        94172  Connecting the dots: how to use relationship n...   
3300        94135  Connect the dots: how to use relationship netw...   
3914       105357   Object and pedestrian detection solutions evolve   
5178       105394  Object and pedestrian detection warehouse safe...   
528          3230      Gen AI-Powered Reinvention: How APAC can Lead   
568          3232      Generative AI Powered Strategy in APAC Region   
4422       105373  Supply Chain Imperatives 2024 and Beyond - Su

As there are only nine duplicate rows, we can easily review the output above manually to identify the following patterns:

* **Titles:** There are slight differences that do not seem relevant regarding content (for example, with and without an article, or with and without a year).
* **Dates:** Publication dates vary, sometimes differing by several months.
* **Domains:** The source domain remains consistently identical across all duplicates.
* **URLs:** Links vary occasionally, typically reflecting different regional paths or language preferences.
* **Tags:** The assigned tags are always identical.


Since the duplicated content consistently originates from the same source domain, we have decided to retain only the earliest published instance of each text. This approach ensures we capture the relevant information precisely when it first emerged.

In [12]:
df['date'] = pd.to_datetime(df['date'])

df_sorted = df.sort_values(by='date', ascending=True)
df_unique = df_sorted.drop_duplicates(subset = 'content', keep = 'first')

df_unique.head()

,Unnamed: 0,title,date,content,domain,url,tags
0,88571,Ricoh to provide customer support for Agility ...,2024-09-11,['The Digit humanoid could work in distributio...,therobotreport,https://www.therobotreport.com/ricoh-provides-...,"['Robotics', 'Video', 'BostonDynamics']"
26,70996,AI Design Trends to Watch in 2025: From Photor...,2024-09-11,['Artificial Intelligence ( AI) is rapidly tra...,geeksforgeeks,https://www.geeksforgeeks.org/ai-design-trends...,"['Personalisation', 'NaturalLanguageProcessing..."
27,104159,AIs generate more novel and exciting research ...,2024-09-11,['The first statistically significant results ...,newatlas,https://newatlas.com/technology/llm-novel-rese...,"['LargeLanguageModel', 'ChatGPT', 'Anthropic',..."
28,73625,Argonne's HPC/AI User Forum Wrap Up,2024-09-11,['As fans of this publication will already kno...,hpcwire,https://www.hpcwire.com/2024/09/11/argonnes-hp...,"['HighPerformanceComputing', 'GenerativeAI', '..."
29,80150,Top 10: Supply Chain Optimisation Strategies,2024-09-11,['Supply chain optimisation has never been mor...,supplychaindigital,https://www.supplychaindigital.com/top10/top-1...,"['Finetuning', 'Traceable', 'Accountability', ..."


In [13]:
df_unique.shape

(16518, 7)

In [14]:
df_unique_content_duplicate = df_unique[df_unique.duplicated(subset ='content')]
print(f"Number of duplicate content:{len(df_unique_content_duplicate)}")

Number of duplicate content:0


We successfully removed duplicate rows. Checking for shape confirmed that 9 rows were removed as well as checking for duplicate content again.

### Handling missing values

Firstly, we check for missing values. We double check with different methods:
- **NA-Values**: Values that are actually missing and indicated as missing in the data
- **Empty string**: Missing values might not be identified because they are empty strings.
- **Placeholders**: Instead of an empty value, placeholders like "N/A" might be used. We check for the most common ones.
- **Empty Lists**: Tags are saved as lists. We check for empty lists.

In [15]:
# Check for NA-values
missing_values_NA = df_unique.isna()
print(f"Number of NA-values: {missing_values_NA.sum().sum()}")

# Check for empty strings
text_columns = df_unique.select_dtypes(include = ['object', 'string']).columns
empty_strings = df_unique[text_columns].apply(lambda col: col.str.strip() == '')
print(f"Number of empty strings: {empty_strings.sum().sum()}")

# Check for placeholders
placeholders = ['N/A', 'n/a', 'null', 'Null', 'none', 'None', 'unknown', 'Unknown', '-']
placeholder_values = df_unique.isin(placeholders)
print(f"Number of placeholders: {placeholder_values.sum().sum()}")

# Check for empty lists
empty_tags_count = (df_unique['tags'] == '[]').sum()
print("Empty tags count:", empty_tags_count)

Number of NA-values: 0
Number of empty strings: 0
Number of placeholders: 0
Empty tags count: 0


There are no missing values in this data set.

### Normalize Text

We create a function that cleans the text in regards of
- **Removal of html tags**: All content within brackets < > is removed to strip away any residual web formatting.
- **Replacing accented characters**: We use the unidecode library to replace all characters with accents (e.g., "é", "ö") with their closest ASCII equivalent ("e", "o") for better consistency.
- **Casing**: We transform the entire text to lower-case letters to ensure that words like "Tech" and "tech" are treated as the same token.
- **Removal of emojis**: WW remove all emoji-like patterns and special Unicode symbols that do not add semantic value to the text analysis.
- **Removal of irrelevant symbols**: We delete all symbols that are not letters, numbers, or spaces, replacing them with a single space to prevent words from merging.
- **Removal of white space**: We collapse multiple spaces into one and strip leading or trailing white spaces from the text.

For this, we use Regex expressions.

In [16]:
def clean_text(text):
    # Check if value is text
    if not isinstance(text, str):
        return text

    # HTML-tags
    text = re.sub(r'<[^>]+>', '', text)

    # Accented characters
    text = unidecode.unidecode(text)

    # Casing
    text = text.lower()

    # Emoji patterns
    emojis_pattern = re.compile(pattern="["
                        u"\U0001F600-\U0001F64F"
                        u"\U0001F300-\U0001F5FF"
                        u"\U0001F680-\U0001F6FF"
                        u"\U0001F1E0-\U0001F1FF"
                        u"\U00002500-\U00002BEF"
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        u"\U0001f926-\U0001f937"
                        u"\U00010000-\U0010ffff"
                        u"\u2640-\u2642"
                        u"\u2600-\u2B55"
                        u"\u200d"
                        u"\u23cf"
                        u"\u23e9"
                        u"\u231a"
                        u"\ufe0f"
                        u"\u3030"
                    "]+", flags=re.UNICODE)
    text = emojis_pattern.sub(r'', text)

    # Symbols
    text = re.sub(r'[^\w\s]', ' ', text)

    # Spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

We now apply the function to the columns title and content. We leave the columns url and tags untouched because:
- **URL**: This doesn't add any information regarding the trend analysis but the url will not work anymore once the symbols are removed.
- **Tags**: If we remove all symbols at once, two words that are one tag (like "Machine Learning") will be seperated from one another. So we have to normalize it separately.

In [17]:
df_clean = df_unique.copy()

cols_to_clean = ['title', 'content']

for col in cols_to_clean:
    df_clean[col] = df_clean[col].apply(clean_text)

print(df_clean.head())

    Unnamed: 0                                              title       date  \
0        88571  ricoh to provide customer support for agility ... 2024-09-11   
26       70996  ai design trends to watch in 2025 from photore... 2024-09-11   
27      104159  ais generate more novel and exciting research ... 2024-09-11   
28       73625                argonne s hpc ai user forum wrap up 2024-09-11   
29       80150        top 10 supply chain optimisation strategies 2024-09-11   

                                              content              domain  \
0   the digit humanoid could work in distribution ...      therobotreport   
26  artificial intelligence ai is rapidly transfor...       geeksforgeeks   
27  the first statistically significant results ar...            newatlas   
28  as fans of this publication will already know ...             hpcwire   
29  supply chain optimisation has never been more ...  supplychaindigital   

                                                  url  \

We quickly check if there are empty strings now, because there might have been content or titles only consisting of symbols.

In [18]:
empty_after_cleaning = (df_clean[cols_to_clean] == '')
print("Empty Values after Cleaning:", empty_after_cleaning.sum().sum())

Empty Values after Cleaning: 0


Now we handle the Tags separately: Lower casing and transform to list instead of text.

In [19]:
df_clean_tag = df_clean.copy()

# Transform strings into real Python lists
df_clean_tag['tags'] = df_clean_tag['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 3. Explode the list, normalize the text, and group it back
df_clean_tag['tags'] = (
    df_clean_tag['tags']
    .explode()
    .str.lower()
    .str.strip()
    .groupby(level=0)
    .apply(lambda x: [tag for tag in x.tolist() if pd.notna(tag)])
)

# check
print(df_clean_tag['tags'].head())

0                     [robotics, video, bostondynamics]
26    [personalisation, naturallanguageprocessing, c...
27    [largelanguagemodel, chatgpt, anthropic, claud...
28    [highperformancecomputing, generativeai, robot...
29    [finetuning, traceable, accountability, roboti...
Name: tags, dtype: object


Finally, give the Dataframe a short name for further analysis:

In [20]:
df_final = df_clean_tag.copy()

## 1.2 Text Preprocessing

*Done by: XXX  - Checked by: XXX*

### Tokenization

For tokenization, we use spaCy. We only tokenize those columns, that contain text, namely title and content. We do not tokenize the other columns:


*   **Date**: Is not in text format.
*   **Domain**: This is a categorical identifier and does not add value to the analysis of the content.
*   **URL**: This would destroy the functionality of the url.
*   **Tags**: We already transformed the tags into a list of strings. Therefore, they are technically already tokenized in a way that respects the specific terminology. If we actually tokenized these values, we would separate key words belonging together like "machine" "learning".




In [21]:
# load model
nlp = spacy.load("en_core_web_sm")

cols_to_tokenize = ['title', 'content']

df_token = df_final.copy()

Because regular tokenization with spacy took a very long time (we interrupted after 25 minutes), we asked Gemini for a way to reduce the time. Using nlp.pipe as well as turning off tagger, parser, ner and lemmatizer were suggested, resulting in a processing time of approx. 28 minutes.

In [ ]:
# Optimized Tokenization
for col in cols_to_tokenize:
    print(f"Tokenizing {col}...")
    # nlp.pipe takes a list of strings and processes them in batches
    # 'disable' turns off everything we don't need for simple tokens
    df_token[f'{col}_token'] = [
        [token.text for token in doc]
        for doc in nlp.pipe(
            df_token[col].astype(str),
            batch_size=100,
            disable=["tagger", "parser", "ner", "lemmatizer"]
        )
    ]

print("Tokenization complete!")

Tokenizing title...
Tokenizing content...


In [ ]:
print(df_token.head())

### Lemmatization

For this, we equally asked Gemini for a time efficient method. It took approx. 35 minutes.

In [ ]:
# Optimized Lemmatization
for col in cols_to_tokenize:
    print(f"Lemmatizing {col}...")
    df_token[f'{col}_lemma'] = [
        [token.lemma_ for token in doc]
        for doc in nlp.pipe(
            df_token[col].astype(str),
            batch_size=100,
            disable=["parser", "ner"] # parser and ner are the slowest parts
        )
    ]

print("Lemmatization complete!")

###Stop word removal

## 1.3 EDA

*Done by: XXX  - Checked by: XXX*

## 1.4 Entity and Relationship Extraction

*Done by: XXX  - Checked by: XXX*

## 1.5 Knowledge Graph Construction

*Done by: XXX  - Checked by: XXX*

## 1.6 Topic Modeling

*Done by: XXX  - Checked by: XXX*